In [ ]:
import torch
torch.cuda.set_device("cuda:1")
import cv2
import os
import ot
import numpy as np
from PIL import Image
from tqdm import tqdm
import supervision as sv
import matplotlib.pyplot as plt
from groundingdino.util.inference import load_model, load_image, predict, annotate
from torchvision.ops import box_convert,nms,box_iou, box_area

In [ ]:
HOME = os.getcwd()
CONFIG_PATH = os.path.join(HOME, "groundingdino/config/GroundingDINO_SwinT_OGC.py")
WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = os.path.join(HOME, "weights", WEIGHTS_NAME)

%cd {HOME}/GroundingDINO
model = load_model(CONFIG_PATH, WEIGHTS_PATH)

In [ ]:
def read_tiff(path):
    """
    path - Path to the multipage-tiff file
    """
    img = Image.open(path)
    images = []
    for i in range(img.n_frames):
        img.seek(i)
        images.append(np.array(img.convert('RGB')))
    return np.array(images)

images = read_tiff("Your own path for tif read")

In [ ]:
#Change the setting here to load the images for GroundingDINO
Level = "Marine_yolo_10m_only"
Ground_truth_train = "/data/Marine_picture/"+ Level + "/images/train/"
Ground_truth_val = "/data/Marine_picture/"+ Level + "/images/val/"
Label_train = "/data/Marine_picture/"+ Level + "/labels/train/"
Label_val = "/data/Marine_picture/"+ Level + "/labels/val/"
im_file = os.listdir(Ground_truth_train)

In [ ]:
categories={
    0: "chaetognath",
    1: "larval fish",
    2: "Hydromedusa",
    3: "lobate ctenophore",
    4: "Pleurobrachia",
    5: "shrimp",
    6: "Siphonophore",
    7: "stomatopod larva",
    8: "Unknown",
    9: "Thaliac",
    10: "polychaete worm",
    11: "Cumacean",
    12:"ctenophore"
}

In [ ]:
mAP = 0
detector_result = {}
for io in np.arange(0.50, 1.00, 0.05):
    iou_threshold = io
    detector_result[io] = {}
    count = {}
    for i in tqdm(im_file):
        temp = i.split(".jpg")[0]
        label = np.loadtxt(Label_train + temp + ".txt")
        ground_boxes = {}
        if type(label[0]) == np.float64 :
            if label[0] in ground_boxes.keys():
                ground_boxes[label[0]] = np.vstack((ground_boxes[label[0]],label[1:5]))
            else:
                ground_boxes[label[0]] = {}
                ground_boxes[label[0]] = label[1:5]
        else:
            for j in label:
                if j[0] in ground_boxes.keys():
                    ground_boxes[j[0]] = np.vstack((ground_boxes[j[0]],j[1:5]))
                else:
                    ground_boxes[j[0]] = {}
                    ground_boxes[j[0]] = j[1:5]


        #print(i, ground_boxes)
        for key in ground_boxes.keys():
            if categories[key] not in detector_result[io].keys():
                count[categories[key]] = 0
                detector_result[io][categories[key]] = {}
                detector_result[io][categories[key]]['TP'] = 0
                detector_result[io][categories[key]]['FP'] = 0
                detector_result[io][categories[key]]['FN'] = 0
                detector_result[io][categories[key]]['P'] = 0

            count[categories[key]] += 1

            Temp = ground_boxes[key]
            Temp_xyxy =box_convert(boxes=torch.from_numpy(Temp), in_fmt='xywh', out_fmt='xyxy')
            Temp_cxcywh = box_convert(boxes=torch.from_numpy(Temp), in_fmt='xywh', out_fmt='cxcywh')

            classes = categories[key]
            ori_img_path = os.path.join(Ground_truth_train,i)
            if len(Temp.shape) == 1:
                num_truth = 1
            else:
                num_truth = Temp.shape[0]

            TEXT_PROMPT = classes
            BOX_TRESHOLD = 0.05
            TEXT_TRESHOLD = 0.05

            image_source, image = load_image(ori_img_path)

            boxes, logits, phrases,logits_saved = predict(
                model=model,
                image=image,
                caption=TEXT_PROMPT,
                box_threshold=BOX_TRESHOLD,
                text_threshold=TEXT_TRESHOLD
            )



            boxes_nms =box_convert(boxes=boxes, in_fmt='xywh', out_fmt='xyxy')
            index_nms = nms(boxes_nms, logits, 0.1)
            index_nms = index_nms.sort().values
            boxes = boxes[index_nms]


            detected_truth = 0
            detected_false = 0
            for l in boxes:
                if l[3] >= 0.8: continue
                box_xyxy =box_convert(boxes=l, in_fmt='xywh', out_fmt='xyxy')
                iou_result = np.asarray(box_iou(box_xyxy.view([1,4]),Temp_xyxy.view([num_truth,4]))).max(axis = 1)  
                if iou_result >= iou_threshold:
                    detected_truth += 1
                    detector_result[io][categories[key]]['TP'] += 1
                else:
                    detected_false += 1
                    detector_result[io][categories[key]]['FP'] += 1

            if num_truth - detected_truth < 0:
                print(num_truth - detected_truth)
            detector_result[io][categories[key]]['FN'] += (num_truth - detected_truth)
            if (detected_truth + detected_false)  != 0:
                detector_result[io][categories[key]]['P'] += detected_truth / (detected_truth + detected_false)        


    for key in detector_result[io].keys():
        detector_result[io][key]['P'] /= count[key]
        
    
    
    temp_mAP = 0
    couts = 0
    for key in detector_result[io]:
        couts += 1
        temp_mAP += detector_result[io][key]['P']
    temp_mAP /= couts
    mAP += temp_mAP
detector_result_10 = detector_result
mAP50_95 = mAP / 10

In [ ]:
def mAP_cal(diction):
    mAP = 0
    length = len(diction.keys())
    for key in diction:
        mAP += diction[key]['P']
    mAP /= length
    return mAP

In [ ]:
#mAP50
mAP_cal(detector_result_10[0.5])

In [ ]:
def mAP50_95(diction):
    mAP = 0
    for key in diction.keys():
        mAP += mAP_cal(diction[key])
    return mAP/10

In [ ]:
#mAP50-95
mAP50_95(detector_result_10)